In [12]:
!pip install gymnasium[atari] stable-baselines3 ale-py autorom
!pip install ale-py
!pip install stable-baselines3[extra]

  Using cached Shimmy-0.2.1-py3-none-any.whl.metadata (2.3 kB)
Using cached Shimmy-0.2.1-py3-none-any.whl (25 kB)
  Attempting uninstall: shimmy
    Found existing installation: Shimmy 1.1.0
    Uninstalling Shimmy-1.1.0:
      Successfully uninstalled Shimmy-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.16.11 requires shimmy>=1.2.1, but you have shimmy 0.2.1 which is incompatible.
  Using cached Shimmy-1.1.0-py3-none-any.whl.metadata (3.3 kB)
Using cached Shimmy-1.1.0-py3-none-any.whl (37 kB)
  Attempting uninstall: shimmy
    Found existing installation: Shimmy 0.2.1
    Uninstalling Shimmy-0.2.1:
      Successfully uninstalled Shimmy-0.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environmen

In [13]:
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import BaseCallback
import numpy as np
import torch
import ale_py

In [14]:
gym.register_envs(ale_py)

In [15]:
class LoggingCallback(BaseCallback):
    def __init__(self, verbose=0):
        # Initialize the base callback with verbosity level
        super(LoggingCallback, self).__init__(verbose)
        # List to store episode rewards
        self.episode_rewards = []
        # List to store episode lengths
        self.episode_lengths = []

    def _on_step(self) -> bool:
        # Check if the episode is done
        if self.locals.get("dones"):
            # Retrieve episode information from the environment
            episode_info = self.locals.get("infos")[0].get("episode", {})
            # Store the reward of the episode
            self.episode_rewards.append(episode_info.get("r", 0))
            # Store the length of the episode
            self.episode_lengths.append(episode_info.get("l", 0))
        return True  # Continue training


In [16]:
env = gym.make("ALE/Boxing-v5", render_mode="rgb_array")

In [17]:
hyperparams = {
    "learning_rate": 1e-4,
    "gamma": 0.99,
    "batch_size": 32,
    "exploration_initial_eps": 1.0,
    "exploration_final_eps": 0.05,
    "exploration_fraction": 0.1
}


In [18]:
model = DQN(
    "CnnPolicy",
    env,
    learning_rate=hyperparams["learning_rate"],
    gamma=hyperparams["gamma"],
    batch_size=hyperparams["batch_size"],
    exploration_initial_eps=hyperparams["exploration_initial_eps"],
    exploration_final_eps=hyperparams["exploration_final_eps"],
    exploration_fraction=hyperparams["exploration_fraction"],
    verbose=1
)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/buffers.py:231: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 201.62GB > 31.81GB
  warnings.warn(


In [19]:
total_timesteps = 100000  # Adjust as needed
logging_callback = LoggingCallback()
model.learn(total_timesteps=total_timesteps, callback=logging_callback)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.79e+03 |
|    ep_rew_mean      | 0.5      |
|    exploration_rate | 0.321    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 794      |
|    time_elapsed     | 8        |
|    total_timesteps  | 7144     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.79e+03 |
|    ep_rew_mean      | -1.62    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 791      |
|    time_elapsed     | 18       |
|    total_timesteps  | 14288    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.79e+03 |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

In [20]:
# Save the trained model
model.save("dqn2_model.zip")

# Optionally, log some final training details
print("Training complete!")
print("Average Episode Reward:", np.mean(logging_callback.episode_rewards))
print("Average Episode Length:", np.mean(logging_callback.episode_lengths))

Training complete!
Average Episode Reward: -5.0
Average Episode Length: 1786.0
